In [5]:
# !nvidia-smi

In [6]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 592, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 592 (delta 125), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (592/592), 194.79 KiB | 1.62 MiB/s, done.
Resolving deltas: 100% (299/299), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.12 environment at: /usr
Resolved 173 packages in 14.53s
 Downloaded cuproj-cu12
 Downloaded shapely
 Downloaded libkvikio-cu12
 Downloaded datashader
 Downloaded ucx-py-cu12
 Downloaded cudf-cu12
 Downloaded pylibcugraph-cu12
 Downloaded dask
 Downloaded librmm-cu12
 Downloaded cucim-cu12
 Downloaded cuspatial-cu12
 Downloaded rmm-cu12
 Downloaded cuml-cu12
 Downloaded cugraph-cu12
 Downloaded libcuspatial-cu12
 Downloaded pylibcudf-cu12
 Downloaded libraft-cu12
 Downloaded libcudf-cu12
 Downloaded raft-dask-cu12
 Downloaded libcuml-cu12
 Downloaded libcuvs-cu12
 Downloaded

In [7]:
import cudf
cudf.__version__

'25.04.00'

In [8]:
import cuml
cuml.__version__

'25.04.00'

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# 📦 Step 1: Load datasets from Google Drive
import pandas as pd
import numpy as np
import time
import tensorflow as tf

print("🔎 TensorFlow version:", tf.__version__)

# Device Check
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("🚀 Running on TPU!")
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print("🧠 Running on GPU!")
        strategy = tf.distribute.MirroredStrategy()
    else:
        print("💻 Running on CPU!")
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")

raw_df = pd.read_csv('/content/drive/MyDrive/natural_disasters_2024.csv')
geo_df = pd.read_csv('/content/drive/MyDrive/cleaned_disasters.csv')
print("✅ Datasets loaded from Google Drive")

# 🛠️ Step 2: Preprocess
raw_df.columns = raw_df.columns.str.strip()
geo_df.columns = geo_df.columns.str.strip()

# Ensure float types
for col in ['Latitude', 'Longitude', 'Magnitude', 'Severity']:
    if col in geo_df.columns:
        geo_df[col] = geo_df[col].astype(np.float32)
    if col in raw_df.columns:
        raw_df[col] = raw_df[col].astype(np.float32)

print("⚙️ Pandas DataFrames ready")

# 🧠 Step 3: Train TensorFlow DNN
X_train = geo_df[['Latitude', 'Longitude', 'Magnitude']].values
y_train = geo_df['Severity'].values

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Regression output
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

model.save('trained_dnn_model.keras')
print("✅ TensorFlow DNN model trained and saved")

# 📡 Step 4: Simulate streaming data
import cupy as cp

stream_batch = raw_df.sample(n=10000).reset_index(drop=True)

# Randomly generate Latitude, Longitude, Magnitude (Fixed)
cp.random.seed(42)
stream_batch['Latitude'] = cp.random.uniform(-90, 90, size=len(stream_batch)).get()
stream_batch['Longitude'] = cp.random.uniform(-180, 180, size=len(stream_batch)).get()
stream_batch['Magnitude'] = cp.asarray(stream_batch['Magnitude']).astype(cp.float32).get()

X_stream = stream_batch[['Latitude', 'Longitude', 'Magnitude']].values

records = stream_batch.to_dict(orient='records')
success_count = 0
failure_count = 0
send_times = []
total_bytes_sent = 0

start_time = time.time()

for idx, record in enumerate(records):
    try:
        # Predict Severity
        features = np.array([[record['Latitude'], record['Longitude'], record['Magnitude']]])
        pred = model.predict(features, verbose=0)[0][0]
        record['Predicted_Severity'] = float(pred)

        # Simulate sending
        t0 = time.time()
        size = len(str(record).encode('utf-8'))
        time.sleep(0.01)  # Artificial latency
        t1 = time.time()

        send_times.append(t1 - t0)
        total_bytes_sent += size
        success_count += 1
    except Exception as e:
        failure_count += 1

end_time = time.time()

# ⏱️ Step 5: Calculate Metrics
total_time = end_time - start_time
avg_speed = success_count / total_time if total_time > 0 else 0
throughput = total_bytes_sent / total_time if total_time > 0 else 0
min_send = min(send_times) if send_times else 0
max_send = max(send_times) if send_times else 0
mean_send = np.mean(send_times) if send_times else 0
median_send = np.median(send_times) if send_times else 0
std_send = np.std(send_times) if send_times else 0

# 📢 Step 6: Print Metrics
print(f"🔢 Total Records: {len(records)}")
print(f"✅ Successfully Sent: {success_count}")
print(f"❌ Failed Sends: {failure_count}")
print(f"⏱️ Total Time Taken: {total_time:.2f} seconds")
print(f"⚡ Average Send Speed: {avg_speed:.2f} records/second")
print(f"📦 Total Data Sent: {total_bytes_sent / 1024:.2f} KB")
print(f"🚀 Throughput: {throughput:.2f} bytes/second")
print(f"📈 Min Send Time: {min_send:.4f} sec")
print(f"📉 Max Send Time: {max_send:.4f} sec")
print(f"📊 Mean Send Time: {mean_send:.4f} sec")
print(f"📏 Median Send Time: {median_send:.4f} sec")
print(f"📐 Std Dev Send Time: {std_send:.4f} sec")

# 🚨 Step 7: Save critical alerts
final_df = pd.DataFrame(records)
alerts = final_df[final_df['Predicted_Severity'] > 8]

if len(alerts) > 0:
    alerts_path = "/content/critical_alerts.csv"
    alerts.to_csv(alerts_path, index=False)
    print(f"⚠️ Critical alerts saved to {alerts_path}")
    from google.colab import files
    files.download(alerts_path)
else:
    print("✅ No critical alerts found.")


🔎 TensorFlow version: 2.18.0
🧠 Running on GPU!
✅ Datasets loaded from Google Drive
⚙️ Pandas DataFrames ready


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 20.5093
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7390
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6252
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.6535
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6365
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6807
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.6105
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6429
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6558
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6326
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6338
Epoch 12/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6378
Epoch 13/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6145
Epoch 14/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6671
Epoch 15/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>